<h1>Data Preprocessing</h1>
<li>Steps that need to be done after the data arrives in the program
<li>Data may need to be cleaned
<ul>
<li>Remove "bad" data
<li>handle "missing" data 
</ul>
<li>Make sure all data is in the correct format(s)


<h2>NYC 311 Data</h2>
<li>Service requests made to the 311 number</li>
<li>https://data.cityofnewyork.us/dataset/311-Service-Requests-From-2011/fpz8-jqf4
<li>We'll work with a small extract of the data
<ul>
<li>8 columns (out of about 58 in the full dataset)
<li>two months of data (out of 7 years of data
</ul>
<li>Even this is quite a bit!

In [ ]:
datafile = "/Users/hardeepjohar/Documents/Courses/DataAnalyticsFall2017/DataVisualization/nyc_311_data_subset_small.csv"

In [ ]:
import pandas as pd
import numpy as np

<h4><b>read_csv</b>: A pandas function that reads a comma separated file</h4>
read_csv will try to format the data so that it is the correct type and will report any typing problems<br>
It will also look for a header row. 
<br>http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
<p>
read_csv returns a pandas DataFrame

In [ ]:
df = pd.read_csv(datafile)
df

<h4>Let's examine our data</h4>

In [ ]:
df.info()

<h4>Looks like Unique Key really is a unique key and can serve as an index</h4>

In [ ]:
df = pd.read_csv(datafile,index_col='Unique Key')

In [ ]:
df.iloc[:10]

<h4>Columns 4 has mixed types</h4>

<h4>Column 4 is incident zip</h4>
Let's examine it<br>
The unique() function returns unique values in a column

In [ ]:
df['Incident Zip'].unique()

<h4>Some issues</h4>
<li>Sometimes zip is a float, other times it is an str
<li>Zipcodes that are represented as floats and start with 0 are missing the first digit (8817)
<li>Some zipcodes have the 4 digit extension added. Comparison becomes tough
<li>What the heck is zip 0?
<li>What about the missing (nan) values? The ? (question mark)? "UNKNOWN"?

<h2>Clean the data</h2>
<li>Decide what to do with "bad" data ("JFK", "UNKNOWN", etc.). Convert to Nan or delete the record.
<li>Make sure all data in a column is in the correct format (convert floats to strings, get rid of the 4 digit extension)
<li>Decide what to do with missing values (NaNs)

<h3>for "Incident Zip"</h3>
<li>we'll drop rows with NaN or bad data
<li>get rid of the 4 digit extension
<li>remove zips less than 10000 and greater than 12000
<li>Let's write a function that fixes zips
<li>Convert everything to str because zips are not really numbers

In [18]:
def fix_zip(input_zip):
    try:
        input_zip = int(float(input_zip))
    except:
        try:
            input_zip = int(input_zip.split('-')[0])
        except:
            return np.NaN
    if input_zip < 10000 or input_zip > 12000:
        return np.NaN
    return str(input_zip)
        

<h4>And test it</h4>

In [20]:
fix_zip('10817.00')

'10817'

<h3>Apply the function to input zip</h3>
<li>The pandas function <span style="color:blue">apply</span> applies a function to each element in a df column


In [21]:
df['Incident Zip'] = df['Incident Zip'].apply(fix_zip)

In [22]:
df['Incident Zip'].unique()

array([nan, '10001', '11691', '11211', '10027', '10452', '11428', '11101',
       '10075', '11215', '11210', '11231', '11217', '10457', '10033',
       '11209', '11201', '11367', '10029', '10021', '10028', '10034',
       '10032', '10039', '11414', '10461', '11229', '10462', '11223',
       '10023', '10453', '11225', '11219', '10451', '11234', '10014',
       '11354', '11361', '10468', '11233', '10466', '11204', '11413',
       '11224', '11375', '11040', '11232', '11203', '11205', '11434',
       '10011', '10003', '10025', '10013', '10036', '11237', '11355',
       '11368', '10454', '10456', '10463', '11222', '11228', '11216',
       '10128', '11435', '11419', '11358', '11421', '10019', '11238',
       '11213', '11235', '11420', '10038', '11226', '10472', '10016',
       '11221', '11236', '11436', '11214', '11377', '11385', '11365',
       '10312', '11426', '11373', '11218', '10005', '11230', '10026',
       '10473', '10280', '10301', '10309', '10310', '10009', '10002',
       '11433',

<h3>Drop all rows that have zip == Nan</h3>
<li>We don't have to, that's just a choice we're making</li>
<li>Alternatively, we could just give them a special code</li>
<li>Or vary the code depending on some other factor (Borough? department?)

In [23]:
df = df[df['Incident Zip'].notnull()]

In [ ]:
df

<h3>Let's take a look at the columns again</h3>

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 910874 entries, 11 to 971063
Data columns (total 7 columns):
Created Date    910874 non-null object
Closed Date     829425 non-null object
Agency          910874 non-null object
Incident Zip    910874 non-null object
Borough         910874 non-null object
Latitude        887168 non-null float64
Longitude       887168 non-null float64
dtypes: float64(2), object(5)
memory usage: 55.6+ MB


<h3>Closed Date, Latitude and Longitude all have missing values</h3>
<h3>Let's get rid of them</h3>

In [25]:
df = df[(df['Latitude'].notnull()) & (df['Longitude'].notnull())  & (df['Closed Date'].notnull())]

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 806561 entries, 11 to 971063
Data columns (total 7 columns):
Created Date    806561 non-null object
Closed Date     806561 non-null object
Agency          806561 non-null object
Incident Zip    806561 non-null object
Borough         806561 non-null object
Latitude        806561 non-null float64
Longitude       806561 non-null float64
dtypes: float64(2), object(5)
memory usage: 49.2+ MB


<h4>Let's take a look at Borough data</h4>

In [27]:
df['Borough'].unique()

array(['MANHATTAN', 'QUEENS', 'BROOKLYN', 'BRONX', 'STATEN ISLAND',
       'Unspecified'], dtype=object)

<h4>Unspecified</h4>
<li>Could be a problem if our analysis focuses on boroughs

In [28]:
df[df['Borough']=='Unspecified'][['Agency','Incident Zip']]

,Agency,Incident Zip
Unique Key,,
28854,NYPD,10312
29525,NYPD,11368
29559,NYPD,11422
29607,NYPD,10454
48565,NYPD,11209
48572,NYPD,11226
48657,NYPD,11225
48743,NYPD,11373
48781,NYPD,10467


<h3>Possible fixes</h3>
<li>Map zips to boroughs using external data
<li>Map zips to boroughs using data from the data frame
<li>Delve into the data to see whether unspecified makes sense

<h4>So we get to use our groupby function</h4>

In [29]:
df[df['Borough']=='Unspecified'].groupby('Agency').count()

,Created Date,Closed Date,Incident Zip,Borough,Latitude,Longitude
Agency,,,,,,
3-1-1,1,1,1,1,1,1
DHS,67,67,67,67,67,67
DOE,1,1,1,1,1,1
DOF,3,3,3,3,3,3
DOT,13,13,13,13,13,13
DPR,2,2,2,2,2,2
FDNY,1,1,1,1,1,1
NYPD,725,725,725,725,725,725
TLC,6,6,6,6,6,6


<li>Unspecified appears to have a systematic bias toward NYPD
<li>Though only a small proportion of NYPD complaints (see below)
<li>We have to decide whether to keep them or lose them!

In [30]:
nypd_complaints_total = df[df['Agency']=='NYPD']['Borough'].count()
nypd_unspecified = df[(df['Borough']=='Unspecified') & (df['Agency']=="NYPD")]['Borough'].count()
percentage = nypd_unspecified/nypd_complaints_total*100
print("%1.2f%%"%percentage)

0.26%


<h4>For now, we'll get rid of them. Unspecified will be hard to explain!</h4>

In [31]:
df = df[df['Borough'] != 'Unspecified']

<h2>Dealing with time</h2>
<li>Dates and times are best converted to datetime
<li>That way they will be useful for analysis because we can compute timedelta objects

<h2>Digression: The datetime library</h2>

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805742 entries, 11 to 971063
Data columns (total 7 columns):
Created Date    805742 non-null object
Closed Date     805742 non-null object
Agency          805742 non-null object
Incident Zip    805742 non-null object
Borough         805742 non-null object
Latitude        805742 non-null float64
Longitude       805742 non-null float64
dtypes: float64(2), object(5)
memory usage: 49.2+ MB


<h4>And we get to use lambda functions!</h4>

In [33]:
import datetime
df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))

In [34]:
df['Created Date'][0:20]

Unique Key
11   2016-09-01 00:33:42
12   2016-09-01 20:16:24
13   2016-09-01 12:17:00
14   2016-09-01 12:10:22
15   2016-09-01 12:32:32
16   2016-09-01 20:16:24
17   2016-09-01 08:35:00
18   2016-09-01 13:19:42
19   2016-09-01 11:00:00
20   2016-09-01 11:45:00
21   2016-09-01 10:11:45
22   2016-09-01 08:22:53
23   2016-09-01 17:31:04
24   2016-09-01 08:50:41
25   2016-09-01 14:19:21
26   2016-09-01 12:46:35
27   2016-09-01 13:33:58
28   2016-09-01 13:51:10
30   2016-09-01 12:34:41
31   2016-09-01 07:03:00
Name: Created Date, dtype: datetime64[ns]

In [35]:
df['Closed Date'] = df['Closed Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))


In [ ]:
df


<h3>We can create a new column that tracks the time it takes to close a complaint</h3>

In [36]:
df['processing_time'] =  df['Closed Date'] - df['Created Date']

In [37]:
#And look at summary statistics
df['processing_time'].describe()


count                     805742
mean      5 days 00:05:11.538976
std      12 days 06:08:17.201098
min          -134 days +00:00:00
25%              0 days 02:34:46
50%       0 days 21:10:44.500000
75%       4 days 14:29:59.750000
max            148 days 13:10:54
Name: processing_time, dtype: object

In [38]:
df['Created Date'].min()

Timestamp('2016-09-01 00:00:00')

<h4>There is some odd stuff here</h4>
<li>Negative processing time?
<li>Since our data is for five months, a max of 148 days worth checking out

<h3>Let's examine the negative processing time data</h3>

In [39]:
df[df['processing_time']<datetime.timedelta(0,0,0)]

,Created Date,Closed Date,Agency,Incident Zip,Borough,Latitude,Longitude,processing_time
Unique Key,,,,,,,,
3972,2016-09-16 14:24:00,2016-09-15 14:23:00,DOT,10314,STATEN ISLAND,40.597868,-74.140537,-2 days +23:59:00
5223,2016-09-20 14:03:00,2016-09-16 14:03:00,DOT,11220,BROOKLYN,40.630682,-74.010970,-4 days +00:00:00
6084,2016-10-20 11:24:00,2016-10-19 01:24:00,DOT,11412,QUEENS,40.696186,-73.751966,-2 days +14:00:00
6085,2016-10-20 16:40:00,2016-10-19 16:39:00,DOT,10306,STATEN ISLAND,40.580343,-74.103262,-2 days +23:59:00
6086,2016-10-20 12:19:00,2016-10-19 12:18:00,DOT,11209,BROOKLYN,40.634865,-74.026381,-2 days +23:59:00
6115,2016-10-20 12:05:00,2016-10-19 02:05:00,DOT,11208,BROOKLYN,40.681095,-73.873586,-2 days +14:00:00
6396,2016-10-24 10:37:00,2016-10-21 10:37:00,DOT,11691,QUEENS,40.608713,-73.747670,-3 days +00:00:00
6682,2016-10-31 10:26:00,2016-10-28 10:26:00,DOT,11417,QUEENS,40.676871,-73.840344,-3 days +00:00:00
6694,2016-10-31 10:46:00,2016-10-27 10:46:00,DOT,11362,QUEENS,40.765202,-73.738088,-4 days +00:00:00


In [40]:
df[df['processing_time']<datetime.timedelta(0,0,0)].count()

Created Date       6419
Closed Date        6419
Agency             6419
Incident Zip       6419
Borough            6419
Latitude           6419
Longitude          6419
processing_time    6419
dtype: int64

<h4>Possibilities</h4>
<li>Data entry errors
<li>Created Date and Closed Date got switched


<h3>And the large processing times as well</h3>

In [41]:
df[df['processing_time']>datetime.timedelta(148,0,0)]

,Created Date,Closed Date,Agency,Incident Zip,Borough,Latitude,Longitude,processing_time
Unique Key,,,,,,,,
155818,2016-09-01 10:49:06,2017-01-28 00:00:00,DOB,11691,QUEENS,40.597741,-73.775975,148 days 13:10:54
156523,2016-09-01 09:04:14,2017-01-27 14:12:22,DOT,11357,QUEENS,40.791344,-73.827361,148 days 05:08:08


In [42]:
df[df['processing_time']>datetime.timedelta(148,0,0)].count()

Created Date       2
Closed Date        2
Agency             2
Incident Zip       2
Borough            2
Latitude           2
Longitude          2
processing_time    2
dtype: int64

<li>Looks like the upper end makes sense but the negative times don't
<li>Though we need to explore this more, we'll get rid of negative times for now

In [43]:
df = df[df['processing_time']>=datetime.timedelta(0,0,0)]

<h2>Finally, let's write a function that incorporates all our changes</h2>

In [44]:
def read_311_data(datafile):
    import pandas as pd
    import numpy as np
    #Add the fix_zip function
    def fix_zip(input_zip):
        try:
            input_zip = int(float(input_zip))
        except:
            try:
                input_zip = int(input_zip.split('-')[0])
            except:
                return np.NaN
        if input_zip < 10000 or input_zip > 19999:
            return np.NaN
        return str(input_zip)
    
    #Read the file
    df = pd.read_csv(datafile,index_col='Unique Key')
    
    #fix the zip
    df['Incident Zip'] = df['Incident Zip'].apply(fix_zip)
    
    #drop all rows that have any nans in them (note the easier syntax!)
    
    df = df.dropna(how='any')
    
    #get rid of unspecified boroughs
    df = df[df['Borough'] != 'Unspecified']
    
    #Convert times to datetime and create a processing time column
    
    import datetime
    df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['Closed Date'] = df['Closed Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['processing_time'] =  df['Closed Date'] - df['Created Date']
    
    #Finally, get rid of negative processing times and return the final data frame
    
    df = df[df['processing_time']>=datetime.timedelta(0,0,0)]
    
    return df
    

In [45]:
df = read_311_data('nyc_311_data_subset_small.csv')
df.info()

/Users/hardeepjohar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


<class 'pandas.core.frame.DataFrame'>
Int64Index: 799323 entries, 11 to 971063
Data columns (total 8 columns):
Created Date       799323 non-null datetime64[ns]
Closed Date        799323 non-null datetime64[ns]
Agency             799323 non-null object
Incident Zip       799323 non-null object
Borough            799323 non-null object
Latitude           799323 non-null float64
Longitude          799323 non-null float64
processing_time    799323 non-null timedelta64[ns]
dtypes: datetime64[ns](2), float64(2), object(3), timedelta64[ns](1)
memory usage: 54.9+ MB


<h1>Summary</h1>
<li>We read the data using read_csv
<li>Cleaned Incident Zip
<li>Converted Dates to datetime
<li>Created a new column containing processing times


<h3>The data is ready!</h3>